In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#import df via path
path = r'C:\Users\shann\05-2023 Instacart Basket Analysis'

In [3]:
#import customers csv
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [4]:
#import order products merged agg pickle
ords_prods_group = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_aggregated.pkl'))

MemoryError: 

In [5]:
#question 4
#check columns listed in customers.csv
customers.columns


Index(['Unnamed: 0', 'user_id', 'First Name', 'Surnam', 'Gender', 'STATE',
       'Age', 'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [6]:
#check shape
customers.shape

(206209, 11)

In [7]:
#check stats
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    206209 non-null  int64 
 1   user_id       206209 non-null  int64 
 2   First Name    194950 non-null  object
 3   Surnam        206209 non-null  object
 4   Gender        206209 non-null  object
 5   STATE         206209 non-null  object
 6   Age           206209 non-null  int64 
 7   date_joined   206209 non-null  object
 8   n_dependants  206209 non-null  int64 
 9   fam_status    206209 non-null  object
 10  income        206209 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 17.3+ MB


First name has only 194950 non-null entries. 
This would mean we have 11259 empty cells.

In [8]:
#double check and confirm first name colun
customers['First Name'].value_counts(dropna = False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: First Name, Length: 208, dtype: int64

11259 missing first names. Going to leave them blank.

In [9]:
#check descriptive state
customers.describe()

,Unnamed: 0,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000,206209.000000
mean,103104.000000,103105.000000,49.501646,1.499823,-0.591696
std,59527.555167,59527.555167,18.480962,1.118433,73.974701
min,0.000000,1.000000,18.000000,0.000000,-128.000000
25%,51552.000000,51553.000000,33.000000,0.000000,-65.000000
50%,103104.000000,103105.000000,49.000000,1.000000,-1.000000
75%,154656.000000,154657.000000,66.000000,3.000000,64.000000
max,206208.000000,206209.000000,81.000000,3.000000,127.000000


In [10]:
#check columns
customers.head(10)

,Unnamed: 0,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,33
1,1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,-107
2,2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,-16
3,3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,65
4,4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,-74
5,5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,-21
6,6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,50
7,7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,-32
8,8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,-128
9,9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,72


Rename for coherency

In [11]:
#rename surname to last name
customers.rename(columns = {'Surname' : 'Last Name'}, inplace = True)

In [12]:
#rename STATE to State
customers.rename(columns = {'STATE' : 'State'}, inplace = True)

In [13]:
#confirm changes to column names
customers.head()

,Unnamed: 0,user_id,First Name,Surnam,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,33
1,1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,-107
2,2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,-16
3,3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,65
4,4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,-74


In [14]:
#question 5 data quality and consistency check
#check for mixed types
for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (customers[weird]) > 0:
        print(col)

First Name


In [15]:
#check for missing values
customers.isnull().sum()

Unnamed: 0          0
user_id             0
First Name      11259
Surnam              0
Gender              0
State               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

In [16]:
#find duplicates
df_dups = customers[customers.duplicated()]

In [17]:
df_dups

,Unnamed: 0,user_id,First Name,Surnam,Gender,State,Age,date_joined,n_dependants,fam_status,income


no duplicates.

In [18]:
#question 6
#check ords_prods_group 
ords_prods_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 25 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   product_id              int64   
 1   product_name            object  
 2   aisle_id                int64   
 3   department_id           int64   
 4   prices                  float64 
 5   Unnamed: 0_y            int64   
 6   order_id                int64   
 7   user_id                 int64   
 8   eval_set                object  
 9   order_number            int64   
 10  order_dow               int64   
 11  order_military_time     int64   
 12  days_since_prior_order  float64 
 13  add_to_cart_order       int64   
 14  reordered               int64   
 15  _merge                  category
 16  busiest_day             object  
 17  busiest_days            object  
 18  busiest_period_of_day   object  
 19  max_order               int64   
 20  loyalty_flag            object  
 21  mean_p

In [19]:
#check the ords_prods_group df size
ords_prods_group.shape

(32404859, 25)

In [20]:
#check info
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    206209 non-null  int64 
 1   user_id       206209 non-null  int64 
 2   First Name    194950 non-null  object
 3   Surnam        206209 non-null  object
 4   Gender        206209 non-null  object
 5   State         206209 non-null  object
 6   Age           206209 non-null  int64 
 7   date_joined   206209 non-null  object
 8   n_dependants  206209 non-null  int64 
 9   fam_status    206209 non-null  object
 10  income        206209 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 17.3+ MB


In [21]:
#combine the two df using the user_id column
combined = ords_prods_group.merge(customers, on = 'user_id')

In [22]:
#check head
combined.head()

,product_id,product_name,aisle_id,department_id,prices,Unnamed: 0_y,order_id,user_id,eval_set,order_number,...,Unnamed: 0,First Name,Surnam,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,1987,3139998,138,prior,28,...,163461,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,-44
1,1,Chocolate Sandwich Cookies,61,19,5.8,1989,1977647,138,prior,30,...,163461,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,-44
2,907,Premium Sliced Bacon,106,12,20.0,1960,3160996,138,prior,1,...,163461,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,-44
3,907,Premium Sliced Bacon,106,12,20.0,1969,2254091,138,prior,10,...,163461,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,-44
4,1000,Apricots,18,10,12.9,1968,505689,138,prior,9,...,163461,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,-44


In [23]:
#check size
combined.shape

(32404859, 35)

In [24]:
#check basic info
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 35 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   product_id              int64   
 1   product_name            object  
 2   aisle_id                int64   
 3   department_id           int64   
 4   prices                  float64 
 5   Unnamed: 0_y            int64   
 6   order_id                int64   
 7   user_id                 int64   
 8   eval_set                object  
 9   order_number            int64   
 10  order_dow               int64   
 11  order_military_time     int64   
 12  days_since_prior_order  float64 
 13  add_to_cart_order       int64   
 14  reordered               int64   
 15  _merge                  category
 16  busiest_day             object  
 17  busiest_days            object  
 18  busiest_period_of_day   object  
 19  max_order               int64   
 20  loyalty_flag            object  
 21  mean_p

In [25]:
#export pickle
combined.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customer_ords_prods_merged.pkl'))